In [1]:
import sys
import os

# Add the project root (RAMBAM-HEMATOLOGY-DEPARTMENT) to sys.path
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.insert(0, project_root)


In [2]:
from simulation.python_sim import SimFunctions, SimClasses
from simulation.model_parameters import ModelParameters
from simulation.simulation_configuration import SimulationConfiguration

from patients.patient import Patient
from patients.leukemia_patient import LeukemiaPatient
from patients.transplant_patient import TransplantPatient
from patients.other_patient import OtherPatient

from simulation.utils import assign_nurse_station, all_left_department

import pandas as pd
import numpy as np
import scipy.stats as scs

from tqdm.notebook import tqdm_notebook
import logging

In [3]:
# Initializing event calendar
Calendar = SimClasses.EventCalendar()

In [4]:
model_parameters = ModelParameters()
simulation_configuration = SimulationConfiguration()

In [5]:
q_flow_station = SimClasses.Resource()
q_flow_station.SetUnits(1)
q_flow_station_queue = SimClasses.FIFOQueue()

secretary_station = SimClasses.Resource()
secretary_station.SetUnits(2)

secretary_station_queue = SimClasses.FIFOQueue()

nurse_station_1 = SimClasses.Resource()
nurse_station_1.SetUnits(1)
nurse_station_2 = SimClasses.Resource()
nurse_station_2.SetUnits(1)
nurse_station_3 = SimClasses.Resource()
nurse_station_3.SetUnits(1)
nurse_station_4 = SimClasses.Resource()
nurse_station_4.SetUnits(1)
nurse_station_5 = SimClasses.Resource()
nurse_station_5.SetUnits(1)
nurse_station_6 = SimClasses.Resource()
nurse_station_6.SetUnits(1)

nurse_station_1_queue = SimClasses.FIFOQueue()
nurse_station_2_queue = SimClasses.FIFOQueue()
nurse_station_3_queue = SimClasses.FIFOQueue()
nurse_station_4_queue = SimClasses.FIFOQueue()
nurse_station_5_queue = SimClasses.FIFOQueue()
nurse_station_6_queue = SimClasses.FIFOQueue()

leukemia_doctor_1 = SimClasses.Resource()
leukemia_doctor_1.SetUnits(1)
leukemia_doctor_2 = SimClasses.Resource()
leukemia_doctor_2.SetUnits(1)
transplant_doctor_1 = SimClasses.Resource()
transplant_doctor_1.SetUnits(1)
transplant_doctor_2 = SimClasses.Resource()
transplant_doctor_2.SetUnits(1)
transplant_doctor_3 = SimClasses.Resource()
transplant_doctor_3.SetUnits(1)

leukemia_doctor_1_queue = SimClasses.FIFOQueue()
leukemia_doctor_2_queue = SimClasses.FIFOQueue()
transplant_doctor_1_queue = SimClasses.FIFOQueue()
transplant_doctor_2_queue = SimClasses.FIFOQueue()
transplant_doctor_3_queue = SimClasses.FIFOQueue()

In [6]:
q_flow_station_wait_time = SimClasses.DTStat()

secretary_station_wait_time = SimClasses.DTStat()

nurse_station_1_wait_time = SimClasses.DTStat()
nurse_station_2_wait_time = SimClasses.DTStat()
nurse_station_3_wait_time = SimClasses.DTStat()
nurse_station_4_wait_time = SimClasses.DTStat()
nurse_station_5_wait_time = SimClasses.DTStat()
nurse_station_6_wait_time = SimClasses.DTStat()

leukemia_doctor_1_wait_time = SimClasses.DTStat()
leukemia_doctor_2_wait_time = SimClasses.DTStat()

transplant_doctor_1_wait_time = SimClasses.DTStat()
transplant_doctor_2_wait_time = SimClasses.DTStat()
transplant_doctor_3_wait_time = SimClasses.DTStat()

In [7]:
leukemia_doctor_1_complex_patients_total_processing_time = SimClasses.DTStat()
leukemia_doctor_1_regular_patients_total_processing_time = SimClasses.DTStat()
leukemia_doctor_2_complex_patients_total_processing_time = SimClasses.DTStat()
leukemia_doctor_2_regular_patients_total_processing_time = SimClasses.DTStat()
transplant_doctor_1_complex_patients_total_processing_time = SimClasses.DTStat()
transplant_doctor_1_regular_patients_total_processing_time = SimClasses.DTStat()
transplant_doctor_2_complex_patients_total_processing_time = SimClasses.DTStat()
transplant_doctor_2_regular_patients_total_processing_time = SimClasses.DTStat()
transplant_doctor_3_complex_patients_total_processing_time = SimClasses.DTStat()
transplant_doctor_3_regular_patients_total_processing_time = SimClasses.DTStat()
other_patients_total_processing_time = SimClasses.DTStat()

In [8]:
q_flow_station_wait_time_avg = []

secretary_station_wait_time_avg = []

nurse_station_1_wait_time_avg = []
nurse_station_2_wait_time_avg = []
nurse_station_3_wait_time_avg = []
nurse_station_4_wait_time_avg = []
nurse_station_5_wait_time_avg = []
nurse_station_6_wait_time_avg = []

leukemia_doctor_1_wait_time_avg = []
leukemia_doctor_2_wait_time_avg = []

transplant_doctor_1_wait_time_avg = []
transplant_doctor_2_wait_time_avg = []
transplant_doctor_3_wait_time_avg = []

q_flow_station_queue_length_avg = []

secretary_station_queue_length_avg = []

nurse_station_1_queue_length_avg = []
nurse_station_2_queue_length_avg = []
nurse_station_3_queue_length_avg = []
nurse_station_4_queue_length_avg = []
nurse_station_5_queue_length_avg = []
nurse_station_6_queue_length_avg = []

leukemia_doctor_1_queue_length_avg = []
leukemia_doctor_2_queue_length_avg = []

transplant_doctor_1_queue_length_avg = []
transplant_doctor_2_queue_length_avg = []
transplant_doctor_3_queue_length_avg = []

q_flow_station_busy_avg = []

secretary_station_busy_avg = []

nurse_station_1_busy_avg = []
nurse_station_2_busy_avg = []
nurse_station_3_busy_avg = []
nurse_station_4_busy_avg = []
nurse_station_5_busy_avg = []
nurse_station_6_busy_avg = []

leukemia_doctor_1_busy_avg = []
leukemia_doctor_2_busy_avg = []

transplant_doctor_1_busy_avg = []
transplant_doctor_2_busy_avg = []
transplant_doctor_3_busy_avg = []

In [9]:
q_flow_station_queue_length = []

secretary_station_queue_length = []

nurse_station_1_queue_length = []
nurse_station_2_queue_length = []
nurse_station_3_queue_length = []
nurse_station_4_queue_length = []
nurse_station_5_queue_length = []
nurse_station_6_queue_length = []

leukemia_doctor_1_queue_length = []
leukemia_doctor_2_queue_length = []

transplant_doctor_1_queue_length = []
transplant_doctor_2_queue_length = []
transplant_doctor_3_queue_length = []

In [10]:
leukemia_doctor_1_complex_patients_total_processing_time_avg = []
leukemia_doctor_1_regular_patients_total_processing_time_avg = []
leukemia_doctor_2_complex_patients_total_processing_time_avg = []
leukemia_doctor_2_regular_patients_total_processing_time_avg = []
transplant_doctor_1_complex_patients_total_processing_time_avg = []
transplant_doctor_1_regular_patients_total_processing_time_avg = []
transplant_doctor_2_complex_patients_total_processing_time_avg = []
transplant_doctor_2_regular_patients_total_processing_time_avg = []
transplant_doctor_3_complex_patients_total_processing_time_avg = []
transplant_doctor_3_regular_patients_total_processing_time_avg = []
other_patients_total_processing_time_avg = []

### We assume here that all patients begin arriving at 08:00 (time 0 is 07:00 hence why we add 60)
#### Currently we assume that they are all summoned at 20 minute intervals for simplicity

In [11]:
leukemia_doctor_1_number_of_patients = model_parameters.leukemia_doctor_1_number_of_regular_patients + model_parameters.leukemia_doctor_1_number_of_complex_patients
leukemia_doctor_1_patient_arrival_times = [60 + i*20 for i in range(leukemia_doctor_1_number_of_patients)]
leukemia_doctor_2_number_of_patients = model_parameters.leukemia_doctor_2_number_of_regular_patients + model_parameters.leukemia_doctor_2_number_of_complex_patients
leukemia_doctor_2_patient_arrival_times = [60 + i*20 for i in range(leukemia_doctor_2_number_of_patients)]

transplant_doctor_1_number_of_patients = model_parameters.transplant_doctor_1_number_of_regular_patients + model_parameters.transplant_doctor_1_number_of_complex_patients
transplant_doctor_1_patient_arrival_times = [60 + i*20 for i in range(transplant_doctor_1_number_of_patients)]
transplant_doctor_2_number_of_patients = model_parameters.transplant_doctor_2_number_of_regular_patients + model_parameters.transplant_doctor_2_number_of_complex_patients
transplant_doctor_2_patient_arrival_times = [60 + i*20 for i in range(transplant_doctor_2_number_of_patients)]
transplant_doctor_3_number_of_patients = model_parameters.transplant_doctor_3_number_of_regular_patients + model_parameters.transplant_doctor_3_number_of_complex_patients
transplant_doctor_3_patient_arrival_times = [60 + i*20 for i in range(transplant_doctor_3_number_of_patients)]

other_patients_arrival_times = [60 + i*10 for i in range(model_parameters.number_of_other_patients)]

In [12]:
def create_patient_schedule(arrival_time: int):
    return {'arrival_time': arrival_time,
            'doctor_consultation_time': arrival_time + 120}

def initialize_patients(patient_arrival_times: list[int],
                        doctor_name: str,
                        calendar: SimClasses.EventCalendar):
    leukemia = 'leukemia' in doctor_name
    transplant = 'transplant' in doctor_name
    patients = []
    if leukemia:
        probability_of_visiting_nurse = model_parameters.probability_of_visiting_nurse_leukemia

        if '1' in doctor_name:
            probability_of_complex_patient = model_parameters.leukemia_doctor_1_probability_of_complex_patient
        else:
            probability_of_complex_patient = model_parameters.leukemia_doctor_2_probability_of_complex_patient

        for arrival_time in patient_arrival_times:
            patients.append(LeukemiaPatient(create_patient_schedule(arrival_time),
                                            doctor_name,
                                            probability_of_complex_patient,
                                            probability_of_visiting_nurse))
    elif transplant:
        probability_of_visiting_nurse = model_parameters.probability_of_visiting_nurse_transplant
        if '1' in doctor_name:
            probability_of_complex_patient = model_parameters.transplant_doctor_1_probability_of_complex_patient
        elif '2' in doctor_name:
            probability_of_complex_patient = model_parameters.transplant_doctor_2_probability_of_complex_patient
        else:
            probability_of_complex_patient = model_parameters.transplant_doctor_3_probability_of_complex_patient

        for arrival_time in patient_arrival_times:
            patients.append(TransplantPatient(create_patient_schedule(arrival_time),
                                              doctor_name,
                                              probability_of_complex_patient,
                                              probability_of_visiting_nurse))
    else: #other
        probability_of_visiting_nurse = model_parameters.probability_of_visiting_nurse_other
        probability_of_complex_patient = model_parameters.probability_of_complex_other_patient

        for arrival_time in patient_arrival_times:
            patients.append(OtherPatient(create_patient_schedule(arrival_time),
                                         doctor_name,
                                         probability_of_complex_patient,
                                         probability_of_visiting_nurse))
    
    for patient in patients:
        patient.schedule_arrival(calendar)
        
    return patients

In [13]:
def q_flow_station_start_of_waiting(new_patient : Patient, clock: float):
    q_flow_station_queue.Add(new_patient)
    new_patient.enter_q_flow_queue(clock)
    if q_flow_station.CurrentNumBusy < q_flow_station.NumberOfUnits:
        q_flow_station.Seize(1)
        next_patient = q_flow_station_queue.Remove()
        q_flow_station_wait_time.Record(SimClasses.Clock - next_patient.enter_q_flow_queue_time)
        SimFunctions.SchedulePlus(Calendar, "q_flow_station_service_start", 0, next_patient)

In [14]:
def q_flow_station_service_start(new_patient : Patient):
    q_flow_service_duration = np.random.exponential(model_parameters.q_flow_mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "q_flow_station_service_end", q_flow_service_duration, new_patient)

In [15]:
def q_flow_station_service_end(new_patient: Patient):
    SimFunctions.SchedulePlus(Calendar, "secretary_station_start_of_waiting", 0, new_patient)

    q_flow_station.Free(1)
    if q_flow_station.CurrentNumBusy < q_flow_station.NumberOfUnits and  q_flow_station_queue.NumQueue() > 0:
        q_flow_station.Seize(1)
        next_patient = q_flow_station_queue.Remove()
        q_flow_station_wait_time.Record(SimClasses.Clock - next_patient.enter_q_flow_queue_time)
        SimFunctions.SchedulePlus(Calendar, "q_flow_station_service_start", 0, next_patient)

In [16]:
def secretary_station_start_of_waiting(new_patient: Patient, clock: float):
    secretary_station_queue.Add(new_patient)
    new_patient.enter_secretary_queue(clock)
    if secretary_station.CurrentNumBusy < secretary_station.NumberOfUnits:
        secretary_station.Seize(1)
        next_patient = secretary_station_queue.Remove()
        secretary_station_wait_time.Record(SimClasses.Clock - next_patient.enter_secretary_queue_time)
        SimFunctions.SchedulePlus(Calendar, "secretary_station_service_start", 0, next_patient)

In [17]:
def secretary_station_service_start(new_patient: Patient):
    secretary_station_service_duration = np.random.exponential(model_parameters.secretary_mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "secretary_station_service_end", secretary_station_service_duration, new_patient)

In [18]:
def secretary_station_service_end(new_patient: Patient):
    patient_type = new_patient.get_type()
    has_nurse_appointment = new_patient.visits_nurse
    if has_nurse_appointment:
        nurse_station = assign_nurse_station(patient_type, model_parameters)
        SimFunctions.SchedulePlus(Calendar, f'{nurse_station}_start_of_waiting', 0, new_patient)
    else:
        SimFunctions.SchedulePlus(Calendar, f'{new_patient.doctor_name}_start_of_waiting', 0, new_patient)
    
    secretary_station.Free(1)
    if secretary_station.CurrentNumBusy < secretary_station.NumberOfUnits and secretary_station_queue.NumQueue() > 0:
        secretary_station.Seize(1)
        next_patient = secretary_station_queue.Remove()
        secretary_station_wait_time.Record(SimClasses.Clock - next_patient.enter_secretary_queue_time)
        SimFunctions.SchedulePlus(Calendar, "secretary_station_service_start", 0, next_patient)

In [19]:
def nurse_station_1_start_of_waiting(new_patient: Patient, clock: float):
    nurse_station_1_queue.Add(new_patient)
    new_patient.enter_nurse_queue(clock)
    if nurse_station_1.CurrentNumBusy < nurse_station_1.NumberOfUnits:
        nurse_station_1.Seize(1)
        next_patient = nurse_station_1_queue.Remove()
        nurse_station_1_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_1_service_start", 0, next_patient)

def nurse_station_2_start_of_waiting(new_patient: Patient, clock: float):
    nurse_station_2_queue.Add(new_patient)
    new_patient.enter_nurse_queue(clock)
    if nurse_station_2.CurrentNumBusy < nurse_station_2.NumberOfUnits:
        nurse_station_2.Seize(1)
        next_patient = nurse_station_2_queue.Remove()
        nurse_station_2_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_2_service_start", 0, next_patient)

def nurse_station_3_start_of_waiting(new_patient: Patient, clock: float):
    nurse_station_3_queue.Add(new_patient)
    new_patient.enter_nurse_queue(clock)
    if nurse_station_3.CurrentNumBusy < nurse_station_3.NumberOfUnits:
        nurse_station_3.Seize(1)
        next_patient = nurse_station_3_queue.Remove()
        nurse_station_3_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_3_service_start", 0, next_patient)

def nurse_station_4_start_of_waiting(new_patient: Patient, clock: float):
    nurse_station_4_queue.Add(new_patient)
    new_patient.enter_nurse_queue(clock)
    if nurse_station_4.CurrentNumBusy < nurse_station_4.NumberOfUnits:
        nurse_station_4.Seize(1)
        next_patient = nurse_station_4_queue.Remove()
        nurse_station_4_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_4_service_start", 0, next_patient)

def nurse_station_5_start_of_waiting(new_patient: Patient, clock: float):
    nurse_station_5_queue.Add(new_patient)
    new_patient.enter_nurse_queue(clock)
    if nurse_station_5.CurrentNumBusy < nurse_station_5.NumberOfUnits:
        nurse_station_5.Seize(1)
        next_patient = nurse_station_5_queue.Remove()
        nurse_station_5_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_5_service_start", 0, next_patient)

def nurse_station_6_start_of_waiting(new_patient: Patient, clock: float):
    nurse_station_6_queue.Add(new_patient)
    new_patient.enter_nurse_queue(clock)
    if nurse_station_6.CurrentNumBusy < nurse_station_6.NumberOfUnits:
        nurse_station_6.Seize(1)
        next_patient = nurse_station_6_queue.Remove()
        nurse_station_6_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_6_service_start", 0, next_patient)

In [20]:
def nurse_station_1_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level

    if patient_complexity == "regular":
        mean_service_time = model_parameters.nurse_mean_service_time_regular
    else:
        mean_service_time = model_parameters.nurse_mean_service_time_complex

    service_duration = np.random.exponential(mean_service_time)

    SimFunctions.SchedulePlus(Calendar, "nurse_station_1_service_end", service_duration, new_patient)

def nurse_station_2_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level

    if patient_complexity == "regular":
        mean_service_time = model_parameters.nurse_mean_service_time_regular
    else:
        mean_service_time = model_parameters.nurse_mean_service_time_complex

    service_duration = np.random.exponential(mean_service_time)

    SimFunctions.SchedulePlus(Calendar, "nurse_station_2_service_end", service_duration, new_patient)

def nurse_station_3_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level

    if patient_complexity == "regular":
        mean_service_time = model_parameters.nurse_mean_service_time_regular
    else:
        mean_service_time = model_parameters.nurse_mean_service_time_complex

    service_duration = np.random.exponential(mean_service_time)

    SimFunctions.SchedulePlus(Calendar, "nurse_station_3_service_end", service_duration, new_patient)

def nurse_station_4_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level

    if patient_complexity == "regular":
        mean_service_time = model_parameters.nurse_mean_service_time_regular
    else:
        mean_service_time = model_parameters.nurse_mean_service_time_complex

    service_duration = np.random.exponential(mean_service_time)

    SimFunctions.SchedulePlus(Calendar, "nurse_station_4_service_end", service_duration, new_patient)

def nurse_station_5_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level

    if patient_complexity == "regular":
        mean_service_time = model_parameters.nurse_mean_service_time_regular
    else:
        mean_service_time = model_parameters.nurse_mean_service_time_complex

    service_duration = np.random.exponential(mean_service_time)

    SimFunctions.SchedulePlus(Calendar, "nurse_station_5_service_end", service_duration, new_patient)

def nurse_station_6_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level

    if patient_complexity == "regular":
        mean_service_time = model_parameters.nurse_mean_service_time_regular
    else:
        mean_service_time = model_parameters.nurse_mean_service_time_complex

    service_duration = np.random.exponential(mean_service_time)

    SimFunctions.SchedulePlus(Calendar, "nurse_station_6_service_end", service_duration, new_patient)

In [21]:
def nurse_station_1_service_end(new_patient: Patient):
    nurse_station_1.Free(1)
    doctor_name = new_patient.doctor_name
    SimFunctions.SchedulePlus(Calendar, f"{doctor_name}_start_of_waiting", 0, new_patient)

    if nurse_station_1.CurrentNumBusy < nurse_station_1.NumberOfUnits and nurse_station_1_queue.NumQueue() > 0:
        nurse_station_1.Seize(1)
        next_patient = nurse_station_1_queue.Remove()
        nurse_station_1_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_1_service_start", 0, next_patient)

def nurse_station_2_service_end(new_patient: Patient):
    nurse_station_2.Free(1)
    doctor_name = new_patient.doctor_name
    SimFunctions.SchedulePlus(Calendar, f"{doctor_name}_start_of_waiting", 0, new_patient)

    if nurse_station_2.CurrentNumBusy < nurse_station_2.NumberOfUnits and nurse_station_2_queue.NumQueue() > 0:
        nurse_station_2.Seize(1)
        next_patient = nurse_station_2_queue.Remove()
        nurse_station_2_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_2_service_start", 0, next_patient)

def nurse_station_3_service_end(new_patient: Patient):
    nurse_station_3.Free(1)
    doctor_name = new_patient.doctor_name
    SimFunctions.SchedulePlus(Calendar, f"{doctor_name}_start_of_waiting", 0, new_patient)

    if nurse_station_3.CurrentNumBusy < nurse_station_3.NumberOfUnits and nurse_station_3_queue.NumQueue() > 0:
        nurse_station_3.Seize(1)
        next_patient = nurse_station_3_queue.Remove()
        nurse_station_3_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_3_service_start", 0, next_patient)

def nurse_station_4_service_end(new_patient: Patient):
    nurse_station_4.Free(1)
    doctor_name = new_patient.doctor_name
    SimFunctions.SchedulePlus(Calendar, f"{doctor_name}_start_of_waiting", 0, new_patient)

    if nurse_station_4.CurrentNumBusy < nurse_station_4.NumberOfUnits and nurse_station_4_queue.NumQueue() > 0:
        nurse_station_4.Seize(1)
        next_patient = nurse_station_4_queue.Remove()
        nurse_station_4_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_4_service_start", 0, next_patient)

def nurse_station_5_service_end(new_patient: Patient):
    nurse_station_5.Free(1)
    doctor_name = new_patient.doctor_name
    SimFunctions.SchedulePlus(Calendar, f"{doctor_name}_start_of_waiting", 0, new_patient)

    if nurse_station_5.CurrentNumBusy < nurse_station_5.NumberOfUnits and nurse_station_5_queue.NumQueue() > 0:
        nurse_station_5.Seize(1)
        next_patient = nurse_station_5_queue.Remove()
        nurse_station_5_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_5_service_start", 0, next_patient)

def nurse_station_6_service_end(new_patient: Patient):
    nurse_station_6.Free(1)
    doctor_name = new_patient.doctor_name
    SimFunctions.SchedulePlus(Calendar, f"{doctor_name}_start_of_waiting", 0, new_patient)

    if nurse_station_6.CurrentNumBusy < nurse_station_6.NumberOfUnits and nurse_station_6_queue.NumQueue() > 0:
        nurse_station_6.Seize(1)
        next_patient = nurse_station_6_queue.Remove()
        nurse_station_6_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_6_service_start", 0, next_patient)

In [22]:
def leukemia_doctor_1_start_of_waiting(new_patient: Patient, clock: float):
    leukemia_doctor_1_queue.Add(new_patient)
    new_patient.enter_doctor_queue(clock)
    if leukemia_doctor_1.CurrentNumBusy < leukemia_doctor_1.NumberOfUnits:
        leukemia_doctor_1.Seize(1)
        next_patient = leukemia_doctor_1_queue.Remove()
        leukemia_doctor_1_wait_time.Record(SimClasses.Clock - next_patient.enter_doctor_queue_time)
        SimFunctions.SchedulePlus(Calendar, "leukemia_doctor_1_service_start", 0, next_patient)

def leukemia_doctor_2_start_of_waiting(new_patient: Patient, clock: float):
    leukemia_doctor_2_queue.Add(new_patient)
    new_patient.enter_doctor_queue(clock)
    if leukemia_doctor_2.CurrentNumBusy < leukemia_doctor_2.NumberOfUnits:
        leukemia_doctor_2.Seize(1)
        next_patient = leukemia_doctor_2_queue.Remove()
        leukemia_doctor_2_wait_time.Record(SimClasses.Clock - next_patient.enter_doctor_queue_time)
        SimFunctions.SchedulePlus(Calendar, "leukemia_doctor_2_service_start", 0, next_patient)

def transplant_doctor_1_start_of_waiting(new_patient: Patient, clock: float):
    transplant_doctor_1_queue.Add(new_patient)
    new_patient.enter_doctor_queue(clock)
    if transplant_doctor_1.CurrentNumBusy < transplant_doctor_1.NumberOfUnits:
        transplant_doctor_1.Seize(1)
        next_patient = transplant_doctor_1_queue.Remove()
        transplant_doctor_1_wait_time.Record(SimClasses.Clock - next_patient.enter_doctor_queue_time)
        SimFunctions.SchedulePlus(Calendar, "transplant_doctor_1_service_start", 0, next_patient)

def transplant_doctor_2_start_of_waiting(new_patient: Patient, clock: float):
    transplant_doctor_2_queue.Add(new_patient)
    new_patient.enter_doctor_queue(clock)
    if transplant_doctor_2.CurrentNumBusy < transplant_doctor_2.NumberOfUnits:
        transplant_doctor_2.Seize(1)
        next_patient = transplant_doctor_2_queue.Remove()
        transplant_doctor_2_wait_time.Record(SimClasses.Clock - next_patient.enter_doctor_queue_time)
        SimFunctions.SchedulePlus(Calendar, "transplant_doctor_2_service_start", 0, next_patient)

def transplant_doctor_3_start_of_waiting(new_patient: Patient, clock: float):
    transplant_doctor_3_queue.Add(new_patient)
    new_patient.enter_doctor_queue(clock)
    if transplant_doctor_3.CurrentNumBusy < transplant_doctor_3.NumberOfUnits:
        transplant_doctor_3.Seize(1)
        next_patient = transplant_doctor_3_queue.Remove()
        transplant_doctor_3_wait_time.Record(SimClasses.Clock - next_patient.enter_doctor_queue_time)
        SimFunctions.SchedulePlus(Calendar, "transplant_doctor_3_service_start", 0, next_patient)

def other_doctor_start_of_waiting(new_patient: Patient, clock: float):
    new_patient.enter_doctor_queue(clock)
    #other patients doctor treatment is not included in the model
    SimFunctions.SchedulePlus(Calendar, "process_complete", 0, new_patient)

In [23]:
def leukemia_doctor_1_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level
    if patient_complexity == "regular":
        mean_service_time = model_parameters.leukemia_doctor_1_mean_service_time_regular
    else:
        mean_service_time = model_parameters.leukemia_doctor_1_mean_service_time_complex
    service_duration = np.random.exponential(mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "leukemia_doctor_1_service_end", service_duration, new_patient)

def leukemia_doctor_2_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level
    if patient_complexity == "regular":
        mean_service_time = model_parameters.leukemia_doctor_2_mean_service_time_regular
    else:
        mean_service_time = model_parameters.leukemia_doctor_2_mean_service_time_complex
    service_duration = np.random.exponential(mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "leukemia_doctor_2_service_end", service_duration, new_patient)

def transplant_doctor_1_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level
    if patient_complexity == "regular":
        mean_service_time = model_parameters.transplant_doctor_1_mean_service_time_regular
    else:
        mean_service_time = model_parameters.transplant_doctor_1_mean_service_time_complex
    service_duration = np.random.exponential(mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "transplant_doctor_1_service_end", service_duration, new_patient)

def transplant_doctor_2_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level
    if patient_complexity == "regular":
        mean_service_time = model_parameters.transplant_doctor_2_mean_service_time_regular
    else:
        mean_service_time = model_parameters.transplant_doctor_2_mean_service_time_complex
    service_duration = np.random.exponential(mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "transplant_doctor_2_service_end", service_duration, new_patient)

def transplant_doctor_3_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level
    if patient_complexity == "regular":
        mean_service_time = model_parameters.transplant_doctor_3_mean_service_time_regular
    else:
        mean_service_time = model_parameters.transplant_doctor_3_mean_service_time_complex
    service_duration = np.random.exponential(mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "transplant_doctor_3_service_end", service_duration, new_patient)

In [24]:
def leukemia_doctor_1_service_end(new_patient: Patient):
    leukemia_doctor_1.Free(1)
    SimFunctions.SchedulePlus(Calendar, "process_complete", 0, new_patient)

    if leukemia_doctor_1.CurrentNumBusy < leukemia_doctor_1.NumberOfUnits and leukemia_doctor_1_queue.NumQueue() > 0:
        leukemia_doctor_1.Seize(1)
        next_patient = leukemia_doctor_1_queue.Remove()
        leukemia_doctor_1_wait_time.Record(SimClasses.Clock - next_patient.enter_doctor_queue_time)
        SimFunctions.SchedulePlus(Calendar, "leukemia_doctor_1_service_start", 0, next_patient)

def leukemia_doctor_2_service_end(new_patient: Patient):
    leukemia_doctor_2.Free(1)
    SimFunctions.SchedulePlus(Calendar, "process_complete", 0, new_patient)

    if leukemia_doctor_2.CurrentNumBusy < leukemia_doctor_2.NumberOfUnits and leukemia_doctor_2_queue.NumQueue() > 0:
        leukemia_doctor_2.Seize(1)
        next_patient = leukemia_doctor_2_queue.Remove()
        leukemia_doctor_2_wait_time.Record(SimClasses.Clock - next_patient.enter_doctor_queue_time)
        SimFunctions.SchedulePlus(Calendar, "leukemia_doctor_2_service_start", 0, next_patient)

def transplant_doctor_1_service_end(new_patient: Patient):
    transplant_doctor_1.Free(1)
    SimFunctions.SchedulePlus(Calendar, "process_complete", 0, new_patient)

    if transplant_doctor_1.CurrentNumBusy < transplant_doctor_1.NumberOfUnits and transplant_doctor_1_queue.NumQueue() > 0:
        transplant_doctor_1.Seize(1)
        next_patient = transplant_doctor_1_queue.Remove()
        transplant_doctor_1_wait_time.Record(SimClasses.Clock - next_patient.enter_doctor_queue_time)
        SimFunctions.SchedulePlus(Calendar, "transplant_doctor_1_service_start", 0, next_patient)

def transplant_doctor_2_service_end(new_patient: Patient):
    transplant_doctor_2.Free(1)
    SimFunctions.SchedulePlus(Calendar, "process_complete", 0, new_patient)

    if transplant_doctor_2.CurrentNumBusy < transplant_doctor_2.NumberOfUnits and transplant_doctor_2_queue.NumQueue() > 0:
        transplant_doctor_2.Seize(1)
        next_patient = transplant_doctor_2_queue.Remove()
        transplant_doctor_2_wait_time.Record(SimClasses.Clock - next_patient.enter_doctor_queue_time)
        SimFunctions.SchedulePlus(Calendar, "transplant_doctor_2_service_start", 0, next_patient)

def transplant_doctor_3_service_end(new_patient: Patient):
    transplant_doctor_3.Free(1)
    SimFunctions.SchedulePlus(Calendar, "process_complete", 0, new_patient)

    if transplant_doctor_3.CurrentNumBusy < transplant_doctor_3.NumberOfUnits and transplant_doctor_3_queue.NumQueue() > 0:
        transplant_doctor_3.Seize(1)
        next_patient = transplant_doctor_3_queue.Remove()
        transplant_doctor_3_wait_time.Record(SimClasses.Clock - next_patient.enter_doctor_queue_time)
        SimFunctions.SchedulePlus(Calendar, "transplant_doctor_3_service_start", 0, next_patient)

In [25]:
def process_complete(new_patient: Patient):
    if new_patient.doctor_name == "leukemia_doctor_1":
        if new_patient.complexity_level == "complex":
            leukemia_doctor_1_complex_patients_total_processing_time.Record(SimClasses.Clock - new_patient.arrival_time)
        else:
            leukemia_doctor_1_regular_patients_total_processing_time.Record(SimClasses.Clock - new_patient.arrival_time)
    elif new_patient.doctor_name == "leukemia_doctor_2":
        if new_patient.complexity_level == "complex":
            leukemia_doctor_2_complex_patients_total_processing_time.Record(SimClasses.Clock - new_patient.arrival_time)
        else:
            leukemia_doctor_2_regular_patients_total_processing_time.Record(SimClasses.Clock - new_patient.arrival_time)
    elif new_patient.doctor_name == "transplant_doctor_1":
        if new_patient.complexity_level == "complex":
            transplant_doctor_1_complex_patients_total_processing_time.Record(SimClasses.Clock - new_patient.arrival_time)
        else:
            transplant_doctor_1_regular_patients_total_processing_time.Record(SimClasses.Clock - new_patient.arrival_time)
    elif new_patient.doctor_name == "transplant_doctor_2":
        if new_patient.complexity_level == "complex":
            transplant_doctor_2_complex_patients_total_processing_time.Record(SimClasses.Clock - new_patient.arrival_time)
        else:
            transplant_doctor_2_regular_patients_total_processing_time.Record(SimClasses.Clock - new_patient.arrival_time)
    elif new_patient.doctor_name == "transplant_doctor_3":
        if new_patient.complexity_level == "complex":
            transplant_doctor_3_complex_patients_total_processing_time.Record(SimClasses.Clock - new_patient.arrival_time)
        else:
            transplant_doctor_3_regular_patients_total_processing_time.Record(SimClasses.Clock - new_patient.arrival_time)
    elif new_patient.doctor_name == "other":
        other_patients_total_processing_time.Record(SimClasses.Clock - new_patient.arrival_time)
    else:
        raise ValueError(f"Invalid doctor name: {new_patient.doctor_name}")
    
    new_patient.end_visit()

In [26]:
pbar_outer = tqdm_notebook(total=simulation_configuration.num_epochs, desc="Running Simulation")
for epoch in range(simulation_configuration.num_epochs):
    SimFunctions.SimFunctionsInit(Calendar)
    leukemia_doctor_1_patients = initialize_patients(leukemia_doctor_1_patient_arrival_times, "leukemia_doctor_1", Calendar)
    leukemia_doctor_2_patients = initialize_patients(leukemia_doctor_2_patient_arrival_times, "leukemia_doctor_2", Calendar)
    transplant_doctor_1_patients = initialize_patients(transplant_doctor_1_patient_arrival_times, "transplant_doctor_1", Calendar)
    transplant_doctor_2_patients = initialize_patients(transplant_doctor_2_patient_arrival_times, "transplant_doctor_2", Calendar)
    transplant_doctor_3_patients = initialize_patients(transplant_doctor_3_patient_arrival_times, "transplant_doctor_3", Calendar)
    other_patients = initialize_patients(other_patients_arrival_times, "other", Calendar)
    list_of_patients = [leukemia_doctor_1_patients, leukemia_doctor_2_patients, transplant_doctor_1_patients, transplant_doctor_2_patients, transplant_doctor_3_patients, other_patients]
    
    pbar_inner = tqdm_notebook(total=Calendar.N(), desc=f"Iteration {epoch + 1} in process", leave=False)
    while Calendar.N() > 0 and not all_left_department(list_of_patients):
        NextEvent = Calendar.Remove()
        SimClasses.Clock = NextEvent.EventTime

        if NextEvent.EventType == "q_flow_station_start_of_waiting":
            q_flow_station_queue_length.append(q_flow_station_queue.NumQueue())
            q_flow_station_start_of_waiting(NextEvent.WhichObject, SimClasses.Clock)
        
        elif NextEvent.EventType == "q_flow_station_service_start":
            q_flow_station_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "q_flow_station_service_end":
            q_flow_station_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "secretary_station_start_of_waiting":
            secretary_station_queue_length.append(secretary_station_queue.NumQueue())
            secretary_station_start_of_waiting(NextEvent.WhichObject, SimClasses.Clock)
        
        elif NextEvent.EventType == "secretary_station_service_start":
            secretary_station_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "secretary_station_service_end":
            secretary_station_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_1_start_of_waiting":
            nurse_station_1_queue_length.append(nurse_station_1_queue.NumQueue())
            nurse_station_1_start_of_waiting(NextEvent.WhichObject, SimClasses.Clock)
        
        elif NextEvent.EventType == "nurse_station_1_service_start":
            nurse_station_1_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_1_service_end":
            nurse_station_1_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_2_start_of_waiting":
            nurse_station_2_queue_length.append(nurse_station_2_queue.NumQueue())
            nurse_station_2_start_of_waiting(NextEvent.WhichObject, SimClasses.Clock)
        
        elif NextEvent.EventType == "nurse_station_2_service_start":
            nurse_station_2_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_2_service_end":
            nurse_station_2_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_3_start_of_waiting":
            nurse_station_3_queue_length.append(nurse_station_3_queue.NumQueue())
            nurse_station_3_start_of_waiting(NextEvent.WhichObject, SimClasses.Clock)
        
        elif NextEvent.EventType == "nurse_station_3_service_start":
            nurse_station_3_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_3_service_end":
            nurse_station_3_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_4_start_of_waiting":
            nurse_station_4_queue_length.append(nurse_station_4_queue.NumQueue())
            nurse_station_4_start_of_waiting(NextEvent.WhichObject, SimClasses.Clock)
        
        elif NextEvent.EventType == "nurse_station_4_service_start":
            nurse_station_4_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_4_service_end":
            nurse_station_4_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_5_start_of_waiting":
            nurse_station_5_queue_length.append(nurse_station_5_queue.NumQueue())
            nurse_station_5_start_of_waiting(NextEvent.WhichObject, SimClasses.Clock)
        
        elif NextEvent.EventType == "nurse_station_5_service_start":
            nurse_station_5_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_5_service_end":
            nurse_station_5_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_6_start_of_waiting":
            nurse_station_6_queue_length.append(nurse_station_6_queue.NumQueue())
            nurse_station_6_start_of_waiting(NextEvent.WhichObject, SimClasses.Clock)
        
        elif NextEvent.EventType == "nurse_station_6_service_start":
            nurse_station_6_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_6_service_end":
            nurse_station_6_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "leukemia_doctor_1_start_of_waiting":
            leukemia_doctor_1_queue_length.append(leukemia_doctor_1_queue.NumQueue())
            leukemia_doctor_1_start_of_waiting(NextEvent.WhichObject, SimClasses.Clock)   

        elif NextEvent.EventType == "leukemia_doctor_1_service_start":
            leukemia_doctor_1_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "leukemia_doctor_1_service_end":
            leukemia_doctor_1_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "leukemia_doctor_2_start_of_waiting":
            leukemia_doctor_2_queue_length.append(leukemia_doctor_2_queue.NumQueue())
            leukemia_doctor_2_start_of_waiting(NextEvent.WhichObject, SimClasses.Clock)
        
        elif NextEvent.EventType == "leukemia_doctor_2_service_start":
            leukemia_doctor_2_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "leukemia_doctor_2_service_end":
            leukemia_doctor_2_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "transplant_doctor_1_start_of_waiting":
            transplant_doctor_1_queue_length.append(transplant_doctor_1_queue.NumQueue())
            transplant_doctor_1_start_of_waiting(NextEvent.WhichObject, SimClasses.Clock)
        
        elif NextEvent.EventType == "transplant_doctor_1_service_start":
            transplant_doctor_1_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "transplant_doctor_1_service_end":
            transplant_doctor_1_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "transplant_doctor_2_start_of_waiting":
            transplant_doctor_2_queue_length.append(transplant_doctor_2_queue.NumQueue())
            transplant_doctor_2_start_of_waiting(NextEvent.WhichObject, SimClasses.Clock)
        
        elif NextEvent.EventType == "transplant_doctor_2_service_start":
            transplant_doctor_2_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "transplant_doctor_2_service_end":
            transplant_doctor_2_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "transplant_doctor_3_start_of_waiting":
            transplant_doctor_3_queue_length.append(transplant_doctor_3_queue.NumQueue())
            transplant_doctor_3_start_of_waiting(NextEvent.WhichObject, SimClasses.Clock)
        
        elif NextEvent.EventType == "transplant_doctor_3_service_start":
            transplant_doctor_3_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "transplant_doctor_3_service_end":
            transplant_doctor_3_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "process_complete" or NextEvent.EventType == "other_start_of_waiting":
            #here the process ends for the other patients
            process_complete(NextEvent.WhichObject)
        else:
            logging.info(NextEvent.EventType)
            logging.error("Unknown event type")
            raise ValueError(f"Unknown event type: {NextEvent.EventType}")

        pbar_inner.update(1)
    pbar_inner.close()

    #storing statistics from all DTStat objects
    q_flow_station_wait_time_avg.append(q_flow_station_wait_time.Mean())
    secretary_station_wait_time_avg.append(secretary_station_wait_time.Mean())
    nurse_station_1_wait_time_avg.append(nurse_station_1_wait_time.Mean())
    nurse_station_2_wait_time_avg.append(nurse_station_2_wait_time.Mean())
    nurse_station_3_wait_time_avg.append(nurse_station_3_wait_time.Mean())
    nurse_station_4_wait_time_avg.append(nurse_station_4_wait_time.Mean())
    nurse_station_5_wait_time_avg.append(nurse_station_5_wait_time.Mean())
    nurse_station_6_wait_time_avg.append(nurse_station_6_wait_time.Mean())
    leukemia_doctor_1_wait_time_avg.append(leukemia_doctor_1_wait_time.Mean())
    leukemia_doctor_2_wait_time_avg.append(leukemia_doctor_2_wait_time.Mean())
    transplant_doctor_1_wait_time_avg.append(transplant_doctor_1_wait_time.Mean())
    transplant_doctor_2_wait_time_avg.append(transplant_doctor_2_wait_time.Mean())
    transplant_doctor_3_wait_time_avg.append(transplant_doctor_3_wait_time.Mean())

    leukemia_doctor_1_complex_patients_total_processing_time_avg.append(leukemia_doctor_1_complex_patients_total_processing_time.Mean())
    leukemia_doctor_1_regular_patients_total_processing_time_avg.append(leukemia_doctor_1_regular_patients_total_processing_time.Mean())
    leukemia_doctor_2_complex_patients_total_processing_time_avg.append(leukemia_doctor_2_complex_patients_total_processing_time.Mean())
    leukemia_doctor_2_regular_patients_total_processing_time_avg.append(leukemia_doctor_2_regular_patients_total_processing_time.Mean())
    transplant_doctor_1_complex_patients_total_processing_time_avg.append(transplant_doctor_1_complex_patients_total_processing_time.Mean())
    transplant_doctor_1_regular_patients_total_processing_time_avg.append(transplant_doctor_1_regular_patients_total_processing_time.Mean())
    transplant_doctor_2_complex_patients_total_processing_time_avg.append(transplant_doctor_2_complex_patients_total_processing_time.Mean())
    transplant_doctor_2_regular_patients_total_processing_time_avg.append(transplant_doctor_2_regular_patients_total_processing_time.Mean())
    transplant_doctor_3_complex_patients_total_processing_time_avg.append(transplant_doctor_3_complex_patients_total_processing_time.Mean())
    transplant_doctor_3_regular_patients_total_processing_time_avg.append(transplant_doctor_3_regular_patients_total_processing_time.Mean())
    other_patients_total_processing_time_avg.append(other_patients_total_processing_time.Mean())

    q_flow_station_queue_length_avg.append(q_flow_station_queue.Mean())
    secretary_station_queue_length_avg.append(secretary_station_queue.Mean())
    nurse_station_1_queue_length_avg.append(nurse_station_1_queue.Mean())
    nurse_station_2_queue_length_avg.append(nurse_station_2_queue.Mean())
    nurse_station_3_queue_length_avg.append(nurse_station_3_queue.Mean())
    nurse_station_4_queue_length_avg.append(nurse_station_4_queue.Mean())
    nurse_station_5_queue_length_avg.append(nurse_station_5_queue.Mean())
    nurse_station_6_queue_length_avg.append(nurse_station_6_queue.Mean())
    leukemia_doctor_1_queue_length_avg.append(leukemia_doctor_1_queue.Mean())
    leukemia_doctor_2_queue_length_avg.append(leukemia_doctor_2_queue.Mean())
    transplant_doctor_1_queue_length_avg.append(transplant_doctor_1_queue.Mean())
    transplant_doctor_2_queue_length_avg.append(transplant_doctor_2_queue.Mean())
    transplant_doctor_3_queue_length_avg.append(transplant_doctor_3_queue.Mean())

    q_flow_station_busy_avg.append(q_flow_station.Mean())
    secretary_station_busy_avg.append(secretary_station.Mean())
    nurse_station_1_busy_avg.append(nurse_station_1.Mean())
    nurse_station_2_busy_avg.append(nurse_station_2.Mean())
    nurse_station_3_busy_avg.append(nurse_station_3.Mean())
    nurse_station_4_busy_avg.append(nurse_station_4.Mean())
    nurse_station_5_busy_avg.append(nurse_station_5.Mean())
    nurse_station_6_busy_avg.append(nurse_station_6.Mean())
    leukemia_doctor_1_busy_avg.append(leukemia_doctor_1.Mean())
    leukemia_doctor_2_busy_avg.append(leukemia_doctor_2.Mean())
    transplant_doctor_1_busy_avg.append(transplant_doctor_1.Mean())
    transplant_doctor_2_busy_avg.append(transplant_doctor_2.Mean())
    transplant_doctor_3_busy_avg.append(transplant_doctor_3.Mean())

    pbar_outer.set_description(f"Running Simulation - {epoch+1}/{simulation_configuration.num_epochs}")
    if epoch == 0 and all_left_department(list_of_patients):
        for patient in leukemia_doctor_1_patients:
            patient.print_schedule()
        for patient in leukemia_doctor_2_patients:
            patient.print_schedule()
        for patient in transplant_doctor_1_patients:
            patient.print_schedule()
        for patient in transplant_doctor_2_patients:
            patient.print_schedule()
        for patient in transplant_doctor_3_patients:
            patient.print_schedule()
        for patient in other_patients:
            patient.print_schedule()
    pbar_outer.update(1)

pbar_outer.close()
logging.info("Simulation completed")

Running Simulation:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration 1 in process:   0%|          | 0/101 [00:00<?, ?it/s]


Patient Schedule Summary — Doctor: leukemia_doctor_1, Complexity: complex, Visits Nurse: True
+-----------------------+------------------+---------------+
| Stage                 | Scheduled Time   |   Actual Time |
+=======================+==================+===============+
| Arrival               | 60               |         54.84 |
+-----------------------+------------------+---------------+
| Q-Flow Queue Entry    | -                |          0    |
+-----------------------+------------------+---------------+
| Secretary Queue Entry | -                |          0    |
+-----------------------+------------------+---------------+
| Nurse Queue Entry     | -                |          0    |
+-----------------------+------------------+---------------+
| Doctor Queue Entry    | -                |          0    |
+-----------------------+------------------+---------------+
| End of Visit          | -                |          0    |
+-----------------------+------------------+-------

Iteration 2 in process:   0%|          | 0/101 [00:00<?, ?it/s]

Iteration 3 in process:   0%|          | 0/101 [00:00<?, ?it/s]

...

Iteration 6 in process:   0%|          | 0/101 [00:00<?, ?it/s]

Iteration 7 in process:   0%|          | 0/101 [00:00<?, ?it/s]

Iteration 8 in process:   0%|          | 0/101 [00:00<?, ?it/s]

Iteration 9 in process:   0%|          | 0/101 [00:00<?, ?it/s]

Iteration 10 in process:   0%|          | 0/101 [00:00<?, ?it/s]

In [27]:
logging.info("Saving Simulation Results")
results_df = pd.DataFrame({
    "q_flow_station_wait_time_avg": q_flow_station_wait_time_avg,
    "q_flow_station_queue_length_avg": q_flow_station_queue_length_avg,
    "q_flow_station_busy_avg": q_flow_station_busy_avg,
    "secretary_station_wait_time_avg": secretary_station_wait_time_avg,
    "secretary_station_queue_length_avg": secretary_station_queue_length_avg,
    "secretary_station_busy_avg": secretary_station_busy_avg,
    "nurse_station_1_wait_time_avg": nurse_station_1_wait_time_avg,
    "nurse_station_1_queue_length_avg": nurse_station_1_queue_length_avg,
    "nurse_station_1_busy_avg": nurse_station_1_busy_avg,
    "nurse_station_2_wait_time_avg": nurse_station_2_wait_time_avg,
    "nurse_station_2_queue_length_avg": nurse_station_2_queue_length_avg,
    "nurse_station_2_busy_avg": nurse_station_2_busy_avg,
    "nurse_station_3_wait_time_avg": nurse_station_3_wait_time_avg,
    "nurse_station_3_queue_length_avg": nurse_station_3_queue_length_avg,
    "nurse_station_3_busy_avg": nurse_station_3_busy_avg,
    "nurse_station_4_wait_time_avg": nurse_station_4_wait_time_avg,
    "nurse_station_4_queue_length_avg": nurse_station_4_queue_length_avg,
    "nurse_station_4_busy_avg": nurse_station_4_busy_avg,
    "nurse_station_5_wait_time_avg": nurse_station_5_wait_time_avg,
    "nurse_station_5_queue_length_avg": nurse_station_5_queue_length_avg,
    "nurse_station_5_busy_avg": nurse_station_5_busy_avg,
    "nurse_station_6_wait_time_avg": nurse_station_6_wait_time_avg,
    "nurse_station_6_queue_length_avg": nurse_station_6_queue_length_avg,
    "nurse_station_6_busy_avg": nurse_station_6_busy_avg,
    "leukemia_doctor_1_wait_time_avg": leukemia_doctor_1_wait_time_avg,
    "leukemia_doctor_1_queue_length_avg": leukemia_doctor_1_queue_length_avg,
    "leukemia_doctor_1_busy_avg": leukemia_doctor_1_busy_avg,
    "leukemia_doctor_2_wait_time_avg": leukemia_doctor_2_wait_time_avg,
    "leukemia_doctor_2_queue_length_avg": leukemia_doctor_2_queue_length_avg,
    "leukemia_doctor_2_busy_avg": leukemia_doctor_2_busy_avg,
    "transplant_doctor_1_wait_time_avg": transplant_doctor_1_wait_time_avg,
    "transplant_doctor_1_queue_length_avg": transplant_doctor_1_queue_length_avg,
    "transplant_doctor_1_busy_avg": transplant_doctor_1_busy_avg,
    "transplant_doctor_2_wait_time_avg": transplant_doctor_2_wait_time_avg,
    "transplant_doctor_2_queue_length_avg": transplant_doctor_2_queue_length_avg,
    "transplant_doctor_2_busy_avg": transplant_doctor_2_busy_avg,
    "transplant_doctor_3_wait_time_avg": transplant_doctor_3_wait_time_avg,
    "transplant_doctor_3_queue_length_avg": transplant_doctor_3_queue_length_avg,
    "transplant_doctor_3_busy_avg": transplant_doctor_3_busy_avg,
    "other_patients_total_processing_time_avg": other_patients_total_processing_time_avg,
    "leukemia_doctor_1_complex_patients_total_processing_time_avg": leukemia_doctor_1_complex_patients_total_processing_time_avg,
    "leukemia_doctor_1_regular_patients_total_processing_time_avg": leukemia_doctor_1_regular_patients_total_processing_time_avg,
    "leukemia_doctor_2_complex_patients_total_processing_time_avg": leukemia_doctor_2_complex_patients_total_processing_time_avg,
    "leukemia_doctor_2_regular_patients_total_processing_time_avg": leukemia_doctor_2_regular_patients_total_processing_time_avg,
    "transplant_doctor_1_complex_patients_total_processing_time_avg": transplant_doctor_1_complex_patients_total_processing_time_avg,
    "transplant_doctor_1_regular_patients_total_processing_time_avg": transplant_doctor_1_regular_patients_total_processing_time_avg,
    "transplant_doctor_2_complex_patients_total_processing_time_avg": transplant_doctor_2_complex_patients_total_processing_time_avg,
    "transplant_doctor_2_regular_patients_total_processing_time_avg": transplant_doctor_2_regular_patients_total_processing_time_avg,
    "transplant_doctor_3_complex_patients_total_processing_time_avg": transplant_doctor_3_complex_patients_total_processing_time_avg,
    "transplant_doctor_3_regular_patients_total_processing_time_avg": transplant_doctor_3_regular_patients_total_processing_time_avg,
    })
os.makedirs("results_directory/simulation_multi_queue", exist_ok=True)
results_df.to_csv(f"results_directory/simulation_multi_queue/results_df.csv")
print(f"Simulation Results Saved to results_directory/simulation_multi_queue/results_df.csv")
print("**********************************************************************************")
print("Means of the simulation results:")
print(results_df.mean())
print("**********************************************************************************")
print("99% Confidence Intervals Half-Width")
CI= scs.t.ppf(1-0.01/2, len(results_df)-1)*np.sqrt(results_df.var()/len(results_df))
print(CI)
print("**********************************************************************************")
print("Error:")
print(CI/results_df.mean())
print("**********************************************************************************")
print("Number of epochs required for 99% confidence, half-width 5 minutes for all measures:")
print(max((scs.t.ppf(1-0.01, len(results_df)-1)*np.sqrt(results_df.var()/len(results_df))/5)**2))
print("Number of epochs required for 99% confidence, half-width 10 minutes for all measures:")
print(max((scs.t.ppf(1-0.01/2, len(results_df)-1)*np.sqrt(results_df.var()/len(results_df))/10)**2))

Simulation Results Saved to results_directory/simulation_multi_queue/results_df.csv
**********************************************************************************
Means of the simulation results:
q_flow_station_wait_time_avg                                      199.727784
q_flow_station_queue_length_avg                                     0.012021
q_flow_station_busy_avg                                             0.065073
secretary_station_wait_time_avg                                   200.383516
secretary_station_queue_length_avg                                  0.023815
secretary_station_busy_avg                                          0.265199
nurse_station_1_wait_time_avg                                     316.266788
nurse_station_1_queue_length_avg                                    2.875563
nurse_station_1_busy_avg                                            0.679194
nurse_station_2_wait_time_avg                                     228.333803
nurse_station_2_queue_length_a

In [28]:

test_calendar = SimClasses.EventCalendar()


leukemia_doctor_1_number_of_patients = model_parameters.leukemia_doctor_1_number_of_regular_patients + model_parameters.leukemia_doctor_1_number_of_complex_patients
leukemia_doctor_1_patient_arrival_times = [60 + i*20 for i in range(leukemia_doctor_1_number_of_patients)]
leukemia_doctor_2_number_of_patients = model_parameters.leukemia_doctor_2_number_of_regular_patients + model_parameters.leukemia_doctor_2_number_of_complex_patients
leukemia_doctor_2_patient_arrival_times = [60 + i*20 for i in range(leukemia_doctor_2_number_of_patients)]

transplant_doctor_1_number_of_patients = model_parameters.transplant_doctor_1_number_of_regular_patients + model_parameters.transplant_doctor_1_number_of_complex_patients
transplant_doctor_1_patient_arrival_times = [60 + i*20 for i in range(transplant_doctor_1_number_of_patients)]
transplant_doctor_2_number_of_patients = model_parameters.transplant_doctor_2_number_of_regular_patients + model_parameters.transplant_doctor_2_number_of_complex_patients
transplant_doctor_2_patient_arrival_times = [60 + i*20 for i in range(transplant_doctor_2_number_of_patients)]
transplant_doctor_3_number_of_patients = model_parameters.transplant_doctor_3_number_of_regular_patients + model_parameters.transplant_doctor_3_number_of_complex_patients
transplant_doctor_3_patient_arrival_times = [60 + i*20 for i in range(transplant_doctor_3_number_of_patients)]

other_patients_arrival_times = [60 + i*10 for i in range(model_parameters.number_of_other_patients)]

leukemia_doctor_1_patients = initialize_patients(leukemia_doctor_1_patient_arrival_times, "leukemia_doctor_1", test_calendar)
leukemia_doctor_2_patients = initialize_patients(leukemia_doctor_2_patient_arrival_times, "leukemia_doctor_2", test_calendar)
transplant_doctor_1_patients = initialize_patients(transplant_doctor_1_patient_arrival_times, "transplant_doctor_1", test_calendar)
transplant_doctor_2_patients = initialize_patients(transplant_doctor_2_patient_arrival_times, "transplant_doctor_2", test_calendar)
transplant_doctor_3_patients = initialize_patients(transplant_doctor_3_patient_arrival_times, "transplant_doctor_3", test_calendar)
other_patients = initialize_patients(other_patients_arrival_times, "other", test_calendar)
list_of_patients = [leukemia_doctor_1_patients, leukemia_doctor_2_patients, transplant_doctor_1_patients, transplant_doctor_2_patients, transplant_doctor_3_patients, other_patients]

for patient_list in list_of_patients:
    for patient in patient_list:
        patient.print_schedule()


Patient Schedule Summary — Doctor: leukemia_doctor_1, Complexity: complex, Visits Nurse: True
+-----------------------+------------------+---------------+
| Stage                 | Scheduled Time   |   Actual Time |
+=======================+==================+===============+
| Arrival               | 60               |         52.41 |
+-----------------------+------------------+---------------+
| Q-Flow Queue Entry    | -                |               |
+-----------------------+------------------+---------------+
| Secretary Queue Entry | -                |               |
+-----------------------+------------------+---------------+
| Nurse Queue Entry     | -                |               |
+-----------------------+------------------+---------------+
| Doctor Queue Entry    | -                |               |
+-----------------------+------------------+---------------+
| End of Visit          | -                |               |
+-----------------------+------------------+-------